In [ ]:
import os
import requests
import shutil
from datetime import datetime
from selenium import webdriver
import time
import urllib
from selenium.webdriver.support.ui import Select
import requests
import shutil
from selenium.webdriver.common.action_chains import ActionChains

os.chdir(r'C:\Users\ruiw\Desktop\work\Wildfire NH3')
dtime=datetime(2018, 7, 31)
minlat=35.4
maxlat=39.0
minlon=-122.0
maxlon=-116.7

res=0.5 #0.5 for 125m, 1 for 250m

#convert to float type
minlat=float(minlat)
minlon=float(minlon)
maxlat=float(maxlat)
maxlon=float(maxlon)
minlatstr=str(minlat).ljust(18,'0')
maxlatstr=str(maxlat).ljust(18,'0')
minlonstr=str(minlon).ljust(18,'0')
maxlonstr=str(maxlon).ljust(18,'0')
yearstr=str(dtime.year)
monthstr=str(dtime.month).rjust(2,'0')
daystr=str(dtime.day).rjust(2,'0')

rlink='https://worldview.earthdata.nasa.gov/?v='+minlonstr+','+minlatstr+','+maxlonstr+','+maxlatstr+'&ics=true&ici=3&icd=2&l=MODIS_Terra_Thermal_Anomalies_Night,MODIS_Terra_Thermal_Anomalies_Day,VIIRS_NOAA20_Thermal_Anomalies_375m_Night(hidden),VIIRS_NOAA20_Thermal_Anomalies_375m_Day(hidden),MODIS_Aqua_Thermal_Anomalies_Night,MODIS_Aqua_Thermal_Anomalies_Day,VIIRS_SNPP_Thermal_Anomalies_375m_Night(hidden),VIIRS_SNPP_Thermal_Anomalies_375m_Day(hidden),Reference_Labels_15m,Reference_Features_15m(hidden),Coastlines_15m,VIIRS_SNPP_CorrectedReflectance_TrueColor(hidden),MODIS_Aqua_CorrectedReflectance_TrueColor(hidden),MODIS_Terra_CorrectedReflectance_TrueColor&lg=false&t=2018-07-31-T04%3A00%3A00Z'
refreshrate = 5
driver = webdriver.Chrome(executable_path=r'C:\Users\ruiw\Downloads\chromedriver_win32\chromedriver.exe')
driver.get(rlink)
driver.maximize_window()
time.sleep(refreshrate)
#refresh the page to get adjusted lat & long
driver.refresh()

#wait and click the snapshot button
time.sleep(refreshrate)
driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]').click()

#crop the image
dragw = driver.find_element_by_class_name('ReactCrop__drag-bar.ord-w')
actionw=ActionChains(driver)
actionw.drag_and_drop_by_offset(dragw, -539, 0).perform()

drage = driver.find_element_by_class_name('ReactCrop__drag-bar.ord-e')
actione=ActionChains(driver)
actione.drag_and_drop_by_offset(drage, 539, 0).perform()

dragn = driver.find_element_by_class_name('ReactCrop__drag-bar.ord-n')
actionn=ActionChains(driver)
actionn.drag_and_drop_by_offset(dragn, 0, -180).perform()

drags = driver.find_element_by_class_name('ReactCrop__drag-bar.ord-s')
actions=ActionChains(driver)
actions.drag_and_drop_by_offset(drags, 0, 180).perform()

#select the res
s1=Select(driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div/div/div[2]/div/div[2]/select'))
s1.select_by_value(str(res))
driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div/div/div[2]/div/div[5]/div[6]/button/span').click()

#switch the tab and get the link of the image
window_handles=driver.window_handles
driver.switch_to.window(window_handles[1])
image_url=driver.current_url
driver.quit()

#read the updated lat&long info from the link
nlatlon=image_url.split('Z&BBOX=')[1].split('&CRS=EPSG')[0].split(',')
nminlat=float(nlatlon[0])
nminlon=float(nlatlon[1])
nmaxlat=float(nlatlon[2])
nmaxlon=float(nlatlon[3])

#download the image
filename=yearstr+monthstr+daystr+'_'+str(0.5*(nminlat+nmaxlat))+'_'+str(0.5*(nminlon+nmaxlon))+'.jpg'

r = requests.get(image_url, stream = True)
if r.status_code == 200:
    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
    r.raw.decode_content = True
    
    # Open a local file with wb ( write binary ) permission.
    with open(filename,'wb') as f:
        shutil.copyfileobj(r.raw, f)
        
    print('Image sucessfully Downloaded: ',filename)
else:
    print('Image Couldn\'t be retreived')